In [1]:
#Тест Непараметрические критерии
import math
import numpy as np
import pandas as pd

#Вопрос 1 - гипотеза о равенстве средних двух независимых выборок
#Вопрос 2 - H0: Fx1(x) = Fx2(x) против H1 ... X+delta
#Вопрос 3 - Критерий Стьюдента???

In [3]:
#Вопрос 4 - Лейкоцитарная лимфома
#H0 = med=200
obs =     np.array([  49,  58,  75, 110, 112, 132, 151, 276, 281, 362])
obs_med = np.array([-151,-142,-125, -90, -88, -68, -49,  76,  81, 162])
ranks =   np.array([   1,   8,    7,   6,  5,   2,   1,   3,   4,  10])

def sign(x): return -1 if x<0 else 1
s = 0
for i in range(10):
    s += ranks[i]*sign(obs_med[i])

from statsmodels.stats.descriptivestats import sign_test
from statsmodels.stats.weightstats import zconfint
from scipy import stats

stats.wilcoxon(obs_med)

WilcoxonResult(statistic=17.0, pvalue=0.2845026979112075)

In [18]:
#Вопрос 5 Лесозаготовки на Борнео
a = np.array([22,22,15,13,19,19,18,20,21,13,13,15])
b = np.array([17,18,18,15,12,4,14,15,10])

from scipy.stats import mannwhitneyu
stats.mannwhitneyu(a, b)


MannwhitneyuResult(statistic=27.0, pvalue=0.02900499272087373)

In [93]:
#Вопрос 6 Челленджер
np.random.seed(0)

df = pd.read_csv("challenger.txt", delimiter="\t", index_col="Date", parse_dates=True)
temp_noincident = np.array(df[df["Incident"] == 0]["Temperature"].tolist())
temp_incident = np.array(df[df["Incident"] == 1]["Temperature"].tolist())

def get_bootstrap_values(data, n_samples):
    indices = np.random.randint(0, len(data), (n_samples, len(data)))
    samples = data[indices]
    return samples

def stat_intervals(stat, alpha):
    boundaries = np.percentile(stat, [100 * alpha / 2, 100 * (1 - alpha / 2)])
    return boundaries

alpha = 0.05

b1 = get_bootstrap_values(temp_noincident, 1000)
b2 = get_bootstrap_values(temp_incident, 1000)

b1m = list(map(np.mean, b1))
b2m = list(map(np.mean, b2))

delta_median_scores = list(map(lambda x: x[0] - x[1], zip(b1m, b2m)))
stat_intervals(delta_median_scores, alpha)

array([1.42299107, 7.93861607])

In [90]:
#Челленджер перестановочным критерием
np.random.seed(0)

def permutation_t_stat_ind(sample1, sample2):
    return np.mean(sample1) - np.mean(sample2)

def get_random_combinations(n1, n2, max_combinations):
    index = list(range(n1 + n2))
    indices = set([tuple(index)])
    for i in range(max_combinations - 1):
        np.random.shuffle(index)
        indices.add(tuple(index))
    return [(index[:n1], index[n1:]) for index in indices]

def permutation_zero_dist_ind(sample1, sample2, max_combinations = None):
    joined_sample = np.hstack((sample1, sample2))
    n1 = len(sample1)
    n = len(joined_sample)
    
    if max_combinations:
        indices = get_random_combinations(n1, len(sample2), max_combinations)
    else:
        indices = [(list(index), filter(lambda i: i not in index, range(n))) \
                    for index in itertools.combinations(range(n), n1)]
    
    distr = [joined_sample[list(i[0])].mean() - joined_sample[list(i[1])].mean() \
             for i in indices]
    return distr

def permutation_test(sample, mean, max_permutations = None, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    t_stat = permutation_t_stat_ind(sample, mean)
    
    zero_distr = permutation_zero_dist_ind(sample, mean, max_permutations)
    
    if alternative == 'two-sided':
        return sum([1. if abs(x) >= abs(t_stat) else 0. for x in zero_distr]) / len(zero_distr)
    
    if alternative == 'less':
        return sum([1. if x <= t_stat else 0. for x in zero_distr]) / len(zero_distr)

    if alternative == 'greater':
        return sum([1. if x >= t_stat else 0. for x in zero_distr]) / len(zero_distr)
    

permutation_test(temp_incident, temp_noincident, max_permutations = 10000)

#0.0057, 0.007

0.0057

In [59]:
index = list(range(6))
tuple(index)

(0, 1, 2, 3, 4, 5)